In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=55488cf4c9e3f4f2753903b1d785f589ec68272c487443c2dad5a795e2534396
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
import folium
import wikipedia as wiki
import re
import pandas as pd
import requests
import urllib.request
import json
import time
from bs4 import BeautifulSoup
from pprint import pprint
from datetime import date

In [ ]:
url = "http://api.open-notify.org/iss-now.json"
response = urllib.request.urlopen(url,timeout=90)
result = json.loads(response.read())
location = result ["iss_position"]
lat = location ["latitude"]
lon = location ["longitude"]
lat = float(lat)
lon = float (lon)

In [ ]:
print ("\nLatitude: " + str(lat))
print ("\nLatitude: " + str(lon))

time.sleep(5)

In [ ]:
map = folium.Map(zoom_start = 5, location = [lat,lon])
folium.Marker(location=[lat,lon], icon= folium.Icon(icon="glyphicon-plane", color = "red"),
              popup= folium.Popup("Current"),
              tooltip = "Current ISS location", ).add_to(map)
map

In [ ]:
url = "http://api.open-notify.org/astros.json"
response = urllib.request.urlopen(url,timeout=90)
result = json.loads(response.read())
people = result ['people']

astronauts = []
spacecraft = []
for p in people:
    astronauts.append(p['name'])
    spacecraft.append(p['craft'])

In [ ]:
print(astronauts)

['Oleg Kononenko', 'Nikolai Chub', 'Tracy Caldwell Dyson', 'Matthew Dominick', 'Michael Barratt', 'Jeanette Epps', 'Alexander Grebenkin', 'Butch Wilmore', 'Sunita Williams', 'Li Guangsu', 'Li Cong', 'Ye Guangfu']


In [ ]:
astro_page = []
for astronaut in astronauts:
  try:
    page = wiki.page(astronaut, auto_suggest=False)
    if 'astronaut' in page.summary.lower() or 'taikonaut' in page.summary.lower() or "cosmonaut" in page.summary.lower():
        astro_page.append(page.url)
        print(page.url)
    else:
        page = wiki.page(f"{astronaut} taikonaut")
        astro_page.append(page.url)
        print(page.url)

  except wiki.DisambiguationError as e:
    for option in e.options:
      if "astronaut" in option.lower():
        try:
          page = wiki.page(option)
          astro_page.append(page.url)
          print (page.url)
        except wiki.DisambiguationError:
          continue

  except wiki.PageError:
      print(f"Page not found for {astronaut}.")

https://en.wikipedia.org/wiki/Oleg_Kononenko
https://en.wikipedia.org/wiki/Nikolai_Chub
https://en.wikipedia.org/wiki/Tracy_Caldwell_Dyson
https://en.wikipedia.org/wiki/Matthew_Dominick


/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


https://en.wikipedia.org/wiki/Michael_Barratt_(astronaut)
https://en.wikipedia.org/wiki/Jeanette_Epps
https://en.wikipedia.org/wiki/Alexander_Grebenkin
https://en.wikipedia.org/wiki/Barry_E._Wilmore
https://en.wikipedia.org/wiki/Sunita_Williams
https://en.wikipedia.org/wiki/Li_Guangsu
https://en.wikipedia.org/wiki/Li_Cong_(taikonaut)
https://en.wikipedia.org/wiki/Ye_Guangfu


In [ ]:
for page in astro_page:
  url = page
  r = requests.get (url=url)
  soup = BeautifulSoup(r.content)
  nationality = soup.select("main#content div.shortdescription.nomobile.noexcerpt.noprint.searchaux")
  print (nationality)

[<div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">Turkmen-Russian cosmonaut (born 1964)</div>]
[<div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">Russian cosmonaut (born 1984)</div>]
[<div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">American chemist and NASA astronaut</div>]
[<div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">American test pilot and astronaut</div>]
[<div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">American aerospace medicine physician and astronaut born 1959</div>]
[<div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">American aerospace engineer and NASA astronaut born 1970</div>]
[<div class="shortdescription nomobile noexcerpt noprint searchaux" style="display:none">Russian cosmonaut (born 1982)</div>]
[<div class="shortdescription nomobile no

In [ ]:
img_links = []
img_src_list = []
dob_list = []
nationality_list = []
age_list = []
today = date.today()
for url in astro_page:
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')

    # Extracting Astronauts images
    covers = soup.select("main#content table.biography tbody tr td.infobox-image span a.mw-file-description img")
    img_src_list.extend(img.get('src') for img in covers)

    # Extracting and Manipulating their birth dates
    infobox_table = soup.find('table', class_='infobox biography vcard')
    if infobox_table:
        next_sibling = infobox_table.find_next_sibling()
        while next_sibling and next_sibling.name != 'p':
            next_sibling = next_sibling.find_next_sibling()
        if next_sibling and next_sibling.name == 'p':
            paragraph_text = next_sibling.get_text(strip=True)
            year_match = re.search(r'\b(19|20)\d{2}\b', paragraph_text)
            if year_match:
                birth_year = int(year_match.group(0))
                age = today.year - birth_year
                dob_list.append(birth_year)
                age_list.append(age)
            else:
                dob_list.append("Year not found")
                age_list.append("Age not found")
        else:
            dob_list.append("No paragraph found immediately following the infobox table.")
            age_list.append("Age not found")
    else:
        dob_list.append("Infobox table not found.")
        age_list.append("Age not found")

    # Extract nationality
    shortdescription_div = soup.select("main#content div.shortdescription.nomobile.noexcerpt.noprint.searchaux")
    if shortdescription_div:
        for div in shortdescription_div:
            text = div.get_text(strip=True)
            nationality_match = re.match(r'\b\w+\b', text)
            nationality_list.append(nationality_match.group(0) if nationality_match else "Nationality not found")
    else:
        nationality_list.append("Info not found")

# Ensure all lists are the same length
max_length = max(len(img_src_list), len(dob_list), len(nationality_list), len(age_list))
img_src_list.extend([""] * (max_length - len(img_src_list)))
dob_list.extend([""] * (max_length - len(dob_list)))
nationality_list.extend([""] * (max_length - len(nationality_list)))
age_list.extend([""] * (max_length - len(age_list)))



In [ ]:
for astronaut in astronauts:
    try:
        page = wiki.page(astronaut, auto_suggest=False)
        if 'astronaut' in page.summary.lower() or 'taikonaut' in page.summary.lower() or "cosmonaut" in page.summary.lower():
            astro_page.append(page.url)
            print(page.url)
        else:
            page = wiki.page(f"{astronaut} taikonaut")
            astro_page.append(page.url)
            print(page.url)
    except wiki.DisambiguationError as e:
        for option in e.options:
            if 'astronaut' in option.lower() or 'taikonaut' in option.lower():
                try:
                    page = wiki.page(option)
                    if 'astronaut' in page.summary.lower() or 'taikonaut' in page.summary.lower():
                        astro_page.append(page.url)
                        print(page.url)
                except wiki.DisambiguationError:
                    continue
    except wiki.PageError:
        print(f"Page not found for {astronaut}.")

https://en.wikipedia.org/wiki/Oleg_Kononenko
https://en.wikipedia.org/wiki/Nikolai_Chub
https://en.wikipedia.org/wiki/Tracy_Caldwell_Dyson
https://en.wikipedia.org/wiki/Matthew_Dominick
https://en.wikipedia.org/wiki/Michael_Barratt_(astronaut)
https://en.wikipedia.org/wiki/Jeanette_Epps
https://en.wikipedia.org/wiki/Alexander_Grebenkin
https://en.wikipedia.org/wiki/Barry_E._Wilmore
https://en.wikipedia.org/wiki/Sunita_Williams
https://en.wikipedia.org/wiki/Li_Guangsu
https://en.wikipedia.org/wiki/Li_Cong_(taikonaut)
https://en.wikipedia.org/wiki/Ye_Guangfu


In [ ]:
df = pd.DataFrame({
    "Astronaut Name": astronauts,
    "Spacecraft": spacecraft,
    'Image Source': img_src_list,
    'Year of Birth': dob_list,
    "Age": age_list,
    'Nationality': nationality_list,
    "Wikipedia Page": astro_page
})

print(df)

          Astronaut Name Spacecraft  \
0         Oleg Kononenko        ISS   
1           Nikolai Chub        ISS   
2   Tracy Caldwell Dyson        ISS   
3       Matthew Dominick        ISS   
4        Michael Barratt        ISS   
5          Jeanette Epps        ISS   
6    Alexander Grebenkin        ISS   
7          Butch Wilmore        ISS   
8        Sunita Williams        ISS   
9             Li Guangsu   Tiangong   
10               Li Cong   Tiangong   
11            Ye Guangfu   Tiangong   

                                         Image Source  Year of Birth  Age  \
0   //upload.wikimedia.org/wikipedia/commons/thumb...           1964   60   
1   //upload.wikimedia.org/wikipedia/commons/thumb...           1984   40   
2   //upload.wikimedia.org/wikipedia/commons/thumb...           1969   55   
3   //upload.wikimedia.org/wikipedia/commons/thumb...           1981   43   
4   //upload.wikimedia.org/wikipedia/commons/thumb...           1959   65   
5   //upload.wikimedia.org/wik